# API REQUEST

In [1]:
from selenium import webdriver
import pandas as pd
from dateutil.parser import parse
import datetime

In [8]:
my_driver = "chromedriver.exe"

op = webdriver.ChromeOptions()
op.add_argument('headless')

url = 'https://www.imdb.com/search/title/?genres=action&groups=top_1000&sort=user_rating,desc&view=simple'
driver = webdriver.Chrome(my_driver,options=op)
driver.implicitly_wait(3)
driver.get(url)
content =  driver.find_elements_by_class_name('lister-item')

#collect in dictionary
content_dict = {'title':[],'rating':[],'total_votes':[],'release_date':[],'duration':[],'description':[],'genres':[],'casts':[],'director':[],'filming_dates':[]}


#for each item do:
for tem_i in range(len(content)):
    #if tem_i == 1:
    #    break

    print("====item: "+str(tem_i)+"===")

    #back to base url
    url = 'https://www.imdb.com/search/title/?genres=action&groups=top_1000&sort=user_rating,desc&view=simple'
    driver = webdriver.Chrome(my_driver,options=op)
    driver.implicitly_wait(1)
    driver.get(url)

    #title
    title_year =  content[tem_i].find_element_by_class_name('col-title')
    title = title_year.find_element_by_tag_name('a')
    title = title.text
    content_dict['title'].append( title )

    #rating
    rating =  content[tem_i].find_element_by_class_name('col-imdb-rating')
    rating = rating.find_element_by_tag_name('strong')
    rating = rating.get_attribute("title")
    content_dict['rating'].append( rating.split()[0] )
    content_dict['total_votes'].append( rating.split()[3] )

    #Follow the link of movie i
    image =  content[tem_i].find_element_by_class_name('lister-item-image')
    image_a = image.find_element_by_tag_name('a')
    url = image_a.get_attribute("href")

    driver = webdriver.Chrome(my_driver,options=op)
    driver.implicitly_wait(1)
    driver.get(url)

    #general_info
    general_info = driver.find_element_by_xpath('.//div[@class="TitleMainBelowTheFoldGroup__TitleMainPrimaryGroup-sc-1vpywau-1 btXiqv ipc-page-grid__item ipc-page-grid__item--span-2"]')


    #release date
    release_date = general_info.find_element_by_xpath('.//section[@data-testid="Details"]')
    release_date = release_date.find_element_by_xpath('.//div[@data-testid="title-details-section"]')
    release_date = release_date.find_element_by_xpath('.//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]')
    content_dict['release_date'].append( release_date.text.split('(')[0] )

    #duration
    duration = general_info.find_element_by_xpath('.//section[@data-testid="TechSpecs"]')
    duration = duration.find_element_by_xpath('.//div[@data-testid="title-techspecs-section"]')
    duration = duration.find_element_by_xpath('.//li[@data-testid="title-techspec_runtime"]')
    duration = duration.find_element_by_xpath('.//li[@class="ipc-inline-list__item"]')
    content_dict['duration'].append( duration.text )

    #description
    description = general_info.find_element_by_xpath('.//section[@data-testid="Storyline"]')
    description = description.find_element_by_xpath('.//div[@class="Storyline__StorylineWrapper-sc-1b58ttw-0 iywpty"]')
    description = description.find_element_by_xpath('.//div[@data-testid="storyline-plot-summary"]')
    description = description.find_element_by_xpath('.//div[@class="ipc-html-content ipc-html-content--base"]')
    content_dict['description'].append( description.text )

    #genre
    genre = general_info.find_element_by_xpath('.//section[@data-testid="Storyline"]')
    genre = genre.find_element_by_xpath('.//div[@class="Storyline__StorylineWrapper-sc-1b58ttw-0 iywpty"]')
    genre = genre.find_element_by_xpath('.//li[@data-testid="storyline-genres"]')
    genre = genre.find_elements_by_xpath('.//li[@class="ipc-inline-list__item"]')
    genres = [i.text for i in genre]
    content_dict['genres'].append( genres )

    #cast
    cast = general_info.find_element_by_xpath('.//section[@data-testid="title-cast"]')
    cast = cast.find_element_by_xpath('.//div[@class="ipc-shoveler title-cast__grid"]')
    cast = cast.find_element_by_xpath('.//div[@class="ipc-sub-grid ipc-sub-grid--page-span-2 ipc-sub-grid--wraps-at-above-l ipc-shoveler__grid"]')
    cast = cast.find_elements_by_xpath('.//div[@data-testid="title-cast-item"]')
    casts = [i.find_element_by_xpath('.//a[@data-testid="title-cast-item__actor"]').text for i in cast]
    content_dict['casts'].append( casts )

    #director
    director = general_info.find_element_by_xpath('.//section[@data-testid="title-cast"]')
    director = director.find_element_by_xpath('.//ul[@class="ipc-metadata-list ipc-metadata-list--dividers-all StyledComponents__CastMetaDataList-y9ygcu-10 cbPPkN ipc-metadata-list--base"]')
    director = director.find_elements_by_tag_name('li')[0]
    director = director.find_element_by_xpath('.//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]')
    content_dict['director'].append( director.text )

    #filming dates
    filming_location = general_info.find_element_by_xpath('.//section[@data-testid="Details"]')
    filming_location = filming_location.find_element_by_xpath('.//div[@data-testid="title-details-section"]')
    filming_location = filming_location.find_element_by_xpath('.//li[@data-testid="title-details-filminglocations"]')
    filming_location = filming_location.find_element_by_xpath('.//a[@class="ipc-metadata-list-item__icon-link"]')

    url = filming_location.get_attribute("href")

    driver = webdriver.Chrome(my_driver,options=op)
    driver.implicitly_wait(1)
    driver.get(url)

    filming_location_info = driver.find_element_by_xpath('.//div[@id="wrapper"]')
    filming_location_info = filming_location_info.find_element_by_xpath('.//div[@id="content-2-wide"]')
    filming_location_info = filming_location_info.find_element_by_xpath('.//section[@class="article listo"]')
    filming_location_info = filming_location_info.find_element_by_xpath('.//section[@id="filming_dates"]')
    filming_dates = filming_location_info.find_element_by_xpath('.//li[@class="ipl-zebra-list__item"]')
    content_dict['filming_dates'].append( filming_dates.text )

print("Done")



====item: 0===
Done


In [9]:
df = pd.DataFrame(content_dict)
print(df)

             title rating total_votes    release_date  duration  \
0  The Dark Knight      9   2,421,273  July 18, 2008   2h 32min   

                                         description  \
0  Set within a year after the events of Batman B...   

                             genres  \
0  [Action, Crime, Drama, Thriller]   

                                               casts           director  \
0  [Christian Bale, Heath Ledger, Aaron Eckhart, ...  Christopher Nolan   

                      filming_dates  
0  18 April 2007 - 11 November 2007  
